In [ ]:
import pandas as pd
import numpy as np

file_list = ["0000_man.csv", "0000_woman.csv", "0000_robot.csv","0001_man.csv", "0001_woman.csv", "0001_robot.csv"]

for file_name in file_list:
    input_file = f"{file_name}"
    output_file = f"pNN50_summary_{file_name}"
    
    try:
        df = pd.read_csv(input_file)
        
        df.columns = df.columns.str.strip()
        df.rename(columns={'CharacterStatus': 'Character_status'}, inplace=True)
        
        if 'rr' in df.columns:
            df['rr'] = pd.to_numeric(df['rr'], errors='coerce')
            df['rr'] = df['rr'].interpolate(method='linear')
            df['rr'].fillna(df['rr'].mean(), inplace=True)
            
            # RR_adjusted addnoise
            df['rr_adjusted'] = df['rr'] + np.random.normal(loc=0, scale=26, size=df.shape[0])
            
            #pNN50 cal
            def calculate_pnn50(group):
                rr_diff = group['rr_adjusted'].diff().dropna()
                nn50 = (rr_diff.abs() > 50).sum()
                total_rr = len(rr_diff)
                return (nn50 / total_rr) * 100 if total_rr > 0 else 0
            
            pnn50_results = df.groupby('Character_status').apply(calculate_pnn50)
            rr_counts = df.groupby('Character_status')['rr_adjusted'].count()
            
            pnn50_summary = pd.DataFrame({
                'pNN50': pnn50_results.round(2).astype(str) + ' %',
                'rr_count': rr_counts
            })
            
            pnn50_summary.to_csv(output_file, index=True)
            print(f"Processed and saved: {output_file}")
            print(pnn50_summary)
        else:
            print(f"Error: No 'RR' column found in {input_file}")
    except Exception as e:
        print(f"Failed to process {input_file}: {e}")


Processed and saved: pNN50_summary_0000_man.csv
                    pNN50  rr_count
Character_status                   
LeftHandToFace    16.83 %      2003
LetfHandToBody    16.61 %      2391
RightHandToBody   16.39 %      1953
RightHandToFace   16.94 %      2900
TouchBody         16.37 %      2261
TouchFace          18.0 %      2512
Walk              16.43 %      1851
Processed and saved: pNN50_summary_0000_woman.csv
                    pNN50  rr_count
Character_status                   
LeftHandToFace    17.59 %      1883
LetfHandToBody     17.7 %      2097
RightHandToBody   17.42 %      1918
RightHandToFace    17.4 %      1754
TouchBody         17.15 %      3592
TouchFace         16.51 %      2387
Walk              19.11 %      1660
Processed and saved: pNN50_summary_0000_robot.csv
                    pNN50  rr_count
Character_status                   
LeftHandToFace     18.0 %      2040
LetfHandToBody    16.74 %      1930
RightHandToBody   16.02 %      3109
RightHandToFace    16.9 